In [1]:
from sociaty_newsletter_generator.db import init_db

await init_db()

In [2]:
from sociaty_newsletter_generator.models import Cluster, ClusteringSession, Article
from sociaty_newsletter_generator.models import (
    Cluster,
    ClusteringSession,
    SetOfUniqueArticles,
)

In [3]:
session_id = "669a345da02feb1858343077"
session = await ClusteringSession.get(session_id)
assert session

In [4]:
from main import format_cluster
async def format_clusters(clusters: list[Cluster], articles_per_cluster_limit: int = 10) -> str:
    clusters_str = ""
    for i, cluster in enumerate(clusters, start=1):
        articles = SetOfUniqueArticles(await cluster.get_articles()).limit(20)
        clusters_str += f"<subject_{i}>\n"
        clusters_str += format_cluster(cluster, articles)
        clusters_str += f"</subject_{i}>\n"
    return clusters_str


c:\Users\samue\AppData\Local\pypoetry\Cache\virtualenvs\sociaty-newsletter-generator-brUSjMkx-py3.12\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `init_chat_model` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [5]:
from main import generate_newsletter

clusters = await Cluster.find_many({"session.$id": session.id,
                                    
                                        "evaluation.decision": "include"
                                    }).to_list()

# order clusters by size
clusters = sorted(clusters, key=lambda x: x.articles_count, reverse=True)

# limit to 5 clusters
clusters = clusters[:5]

clusters_str = await format_clusters(clusters)

# crew_output = generate_newsletter(clusters_str, language='fr')
# crew_output.raw

In [6]:
sessions= [
    '6697a91d6720e5b25ef7946b',
    '6697ab4a6720e5b25ef794d9',
    '6697ab766720e5b25ef795e2',
    '6697ad356720e5b25ef79605',
    '6697aec36720e5b25ef796fc',
    '6697b08c6720e5b25ef797e2',
    '66993bf59a3b5803097f77df',
    '669a345da02feb1858343077',
    '669e16804f5d02e9b56027c9',
    '669e1f654042a3dbb35f9484',
    '669f6cbd20d8c0fd9be0259f',
    '669f794b8f71f776cd33b021',

]

In [7]:
from datetime import datetime
from slugify import slugify
from pathlib import Path
from main import format_cluster, generate_newsletter
from sociaty_newsletter_generator.models import SetOfUniqueArticles

for session in sessions:
    session = await ClusteringSession.get(session)
    assert session

    print(f"Generating newsletter for session {session.id}")

# and evaluation.decision == "include"
    clusters = await Cluster.find_many({"session.$id": session.id,
                                        "evaluation.decision": "include"
                                        }).to_list()

    # order clusters by size
    clusters = sorted(clusters, key=lambda x: x.articles_count, reverse=True)

    # limit to 5 clusters
    clusters = clusters[:5]

    clusters_str = await format_clusters(clusters)

    for i, cluster in enumerate(clusters, start=1):
        print(f"- {i}. {cluster.title}")

    output = generate_newsletter(clusters_str,
                                  language='fr')

    now = datetime.now()
    output_dir = Path("output")
    output_dir.mkdir(exist_ok=True)

    file_name = f"{now.strftime('%Y_%m_%d_%H_%M_%S')}_session_{session.id}.md"
    with open(output_dir / file_name, "w", encoding='utf-8') as f:
        f.write(output.raw)
    print(f"Generated {file_name}")


Generating newsletter for session 6697a91d6720e5b25ef7946b
- 1. SK Hynix's $75 Billion Investment Focuses on AI-Enhanced HBM Processors
- 2. Controversy Surrounding AI Training on Web Content Sparks Legal Battles
- 3. Morgan Freeman Condemns Unauthorized AI Voice Imitations
- 4. Apple Working to Integrate AI Features into Vision Pro Headsets
- 5. States Addressing AI Skills Gap in Workforce
Generated 2024_07_30_14_15_44_session_6697a91d6720e5b25ef7946b.md
Generating newsletter for session 6697ab4a6720e5b25ef794d9
- 1. Google's AI-driven operations lead to 48% increase in carbon emissions
- 2. Apple secures observer role on OpenAI board in landmark AI agreement
- 3. Google Pixel 9 to Introduce New 'Google AI' Features
- 4. French Antitrust Regulators Target Nvidia for AI Dominance
- 5. French AI Lab Kyutai Revolutionizes Voice AI with Moshi
Generated 2024_07_30_14_16_09_session_6697ab4a6720e5b25ef794d9.md
Generating newsletter for session 6697ab766720e5b25ef795e2
- 1. OpenAI's 2023 Brea